# AQI to Cigarette Equivalent


The concept of comparing air quality to the number of cigarettes smoked
is a somewhat popular analogy to help people understand the potential health
risks associated with poor air quality. However, the
comparison is a simplification and has its limitations.

One of the most commonly cited analogies comes from a study that equates the PM2.5 exposure from smoking a single cigarette to be approximately equivalent to a PM2.5 concentration of 22 µg/m³ over 24 hours.

References: https://berkeleyearth.org/air-pollution-and-cigarette-equivalence/

<!-- ##**Please save a copy of this notebook (File/Save a copy in Drive) before proceding!** -->

In [5]:
#!pip install ipywidgets requests > /dev/null 2>&1

import requests
import ipywidgets as widgets
from IPython.display import display, clear_output

In [6]:
def fetch_aqi_from_zip(zip_code):
    API_KEY = "DBD2BF2A-95EB-4314-9D02-822859CDF790"  # Embedding the provided API key
    URL = f"https://www.airnowapi.org/aq/observation/zipCode/current/?format=application/json&zipCode={zip_code}&distance=25&API_KEY={API_KEY}"
    response = requests.get(URL)
    data = response.json()

    # Extracting the PM2.5 AQI value
    for entry in data:
        if entry['ParameterName'] == 'PM2.5':
            return entry['AQI']
    return None  # Return None if no PM2.5 data found

In [7]:
def calculate_cigarette_and_AQI_with_activity(AQI_value, hours=1, activity="resting"):
    # Given values for AQI breakpoints and ventilation rates
    breakpoints = {
        "Good": {"Low AQI": 0, "High AQI": 50, "Low Conc": 0.0, "High Conc": 12.0},
        "Moderate": {"Low AQI": 51, "High AQI": 100, "Low Conc": 12.1, "High Conc": 35.4},
        "Unhealthy for Sensitive Groups": {"Low AQI": 101, "High AQI": 150, "Low Conc": 35.5, "High Conc": 55.4},
        "Unhealthy": {"Low AQI": 151, "High AQI": 200, "Low Conc": 55.5, "High Conc": 150.4},
        "Very Unhealthy": {"Low AQI": 201, "High AQI": 300, "Low Conc": 150.5, "High Conc": 250.4},
        "Hazardous": {"Low AQI": 301, "High AQI": 500, "Low Conc": 250.5, "High Conc": 500.4},
    }

    ventilation_rates = {
        "breathing": 7.5,
        "light activity": 22.5,  # average of 15 and 30
        "running/cycling": 60
    }

    # Determine the AQI category
    category = next((k for k, v in breakpoints.items() if v["Low AQI"] <= AQI_value <= v["High AQI"]), None)

    if not category:
        return "Invalid AQI value"

    # Extract values for the determined AQI category
    low_AQI = breakpoints[category]["Low AQI"]
    high_AQI = breakpoints[category]["High AQI"]
    low_concentration = breakpoints[category]["Low Conc"]
    high_concentration = breakpoints[category]["High Conc"]

    # Calculate PM2.5 concentration using the AQI formula
    concentration = ((AQI_value - low_AQI) * (high_concentration - low_concentration) / (high_AQI - low_AQI)) + low_concentration

    # Adjust the concentration based on activity level
    adjusted_concentration = concentration * ventilation_rates[activity] / ventilation_rates["breathing"]

    # Estimate the equivalent number of cigarettes for specified hours of exposure
    cigarette_equivalent = adjusted_concentration * hours / (22 * 24)

    # Create the summary sentences
    AQI_summary = f"The AQI value of {AQI_value} corresponds to a PM2.5 concentration of approximately {concentration:.2f} µg/m³."

    if cigarette_equivalent < 1:
        cigarette_summary = f"A {hours}-hour exposure at this concentration while {activity} is roughly equivalent to smoking about {cigarette_equivalent*100:.2f}% of one cigarette."
    else:
        cigarette_summary = f"A {hours}-hour exposure at this concentration while {activity} is roughly equivalent to smoking about {cigarette_equivalent:.2f} cigarettes."

    return AQI_summary, cigarette_summary

In [12]:
import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import display, HTML


# Dropdown for activity level
activity_widget = widgets.Dropdown(
    options=['breathing', 'light activity', 'running/cycling'],
    value='breathing',
    description='Activity:'
)

# Slider for exposure time
hours_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=24,
    step=1,
    description='Hours:'
)

# Text input for ZIP code
zip_widget = widgets.Text(
    value='',
    placeholder='Enter ZIP code',
    description='ZIP Code:'
)

button = widgets.Button(description="Calculate")
output = widgets.Output()

def on_button_click(b):
    # Fetch AQI data
    AQI_value = fetch_aqi_from_zip(zip_widget.value)

    # Calculate cigarette equivalent
    AQI_summary, cigarette_summary = calculate_cigarette_and_AQI_with_activity(AQI_value, hours_widget.value, activity_widget.value)

    with output:
        clear_output(wait=True)
        display(HTML('<span style="color: red">{}</span>'.format(AQI_summary)))
        display(HTML('<span style="color: red">{}</span>'.format(cigarette_summary)))

button.on_click(on_button_click)

# Display the widgets for user input
display(zip_widget, activity_widget, hours_widget, button, output)

/var/folders/yb/b0lcxvcx4r3g8zfh98l7mbkw0000gq/T/ipykernel_69216/2123377787.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Text(value='', description='ZIP Code:', placeholder='Enter ZIP code')

Dropdown(description='Activity:', options=('breathing', 'light activity', 'running/cycling'), value='breathing…

IntSlider(value=1, description='Hours:', max=24, min=1)

Button(description='Calculate', style=ButtonStyle())

Output()

For the conversion from AQI to PM2.5 concentration:

$$
C = \frac{(AQI - AQI_{low}) \times (C_{high} - C_{low})}{AQI_{high} - AQI_{low}} + C_{low}
$$

Where:
- $C$ is the PM2.5 Concentration in µg/m³ over 1 hour.
- $AQI$ represents the Air Quality Index value.
- $AQI_{low}$ and $AQI_{high}$ are the lower and upper bounds of the AQI range, respectively.
- $C_{low}$ and $C_{high}$ are the PM2.5 concentration values corresponding to $AQI_{low}$ and $AQI_{high}$, respectively.

**For PM2.5 in the "Unhealthy" range (AQI: 151-200):**

- $AQI_{low}$: 151
- $AQI_{high}$: 200
- $C_{low}$: 35.5 µg/m³
- $C_{high}$: 55.4 µg/m³

To convert AQI exposure to the number of smoked cigarettes:

$$
\text{Cigarette Equivalent (1 hour)} = \frac{C}{PM2.5_{cigarette} \times h}
$$

Where:
- $PM2.5_{cigarette}$ represents the PM2.5 concentration equivalent for one cigarette, which is approximately 22 µg/m³ over a 24-hour period.

- \$h$ represents the number of hours in a day, considering continuous exposure, which is 24 hours.






## Ventilation Rates and PM2.5 Exposure during Activities

When exercising, especially during activities like running, the respiratory rate increases, leading to higher inhalation of the surrounding air. This means that the effective exposure to pollutants during exercise can be significantly higher than during periods of rest or light activity.

A common way to quantify this is by looking at the **ventilation rate**, which is the volume of air inhaled (or exhaled) per minute. Here's a rough breakdown:

- **Resting**: 6 to 7.5 liters of air per minute.<sup><a href="#ref1" id="ref1-1">1</a></sup>
- **Light Activity**: 15 to 30 liters of air per minute.<sup><a href="#ref1" id="ref1-2">1</a></sup>
- **Exercising (like running)**: Can be 60 liters per minute or higher, depending on the intensity.<sup><a href="#ref2" id="ref2-1">2</a></sup>

If we make a simplifying assumption that the increase in the volume of air inhaled corresponds linearly to the increase in PM2.5 exposure, we can use these ventilation rates to adjust our calculations.

For instance, if you're running, you might inhale 8-10 times more air (and hence PM2.5) per minute than when you're at rest.

---

**References**:

<a id="ref1"></a>1. West, J. B. (2012). Respiratory physiology: the essentials. Lippincott Williams & Wilkins.

<a id="ref2"></a>2. American College of Sports Medicine. (2013). ACSM's guidelines for exercise testing and prescription. Lippincott Williams & Wilkins.
